# New York City Recycling and Diversion Rates

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from folium import Choropleth

In [5]:
#reading in recycle and diversion file
recycle = pd.read_csv('https://data.cityofnewyork.us/resource/gaq9-z3hz.csv')
recycle.head()

,_zone,district,fiscal_month_number,fiscal_year,month_name,diversion_rate_total_total_recycling_total_waste_,capture_rate_paper_total_paper_max_paper_,capture_rate_mgp_total_mgp_max_mgp_,capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100
0,Brooklyn North,BKN01,10,2019,April,14.687093,44.909160,43.034062,44.146764
1,Brooklyn North,BKN02,10,2019,April,19.950181,34.194020,57.947031,41.213700
2,Brooklyn North,BKN03,10,2019,April,12.164161,33.521557,44.919731,38.155937
3,Brooklyn North,BKN04,10,2019,April,15.541803,35.211361,68.511260,48.750755
4,Brooklyn North,BKN05,10,2019,April,10.051845,22.265430,45.051791,31.530129


In [7]:
#reading in shapefile of community districts
dist = gpd.read_file('Community Districts/geo_export_362b8b30-acf1-46f8-acdc-971fc2ff0c6e.shp')
dist.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   boro_cd     71 non-null     float64 
 1   shape_area  71 non-null     float64 
 2   shape_leng  71 non-null     float64 
 3   geometry    71 non-null     geometry
dtypes: float64(3), geometry(1)
memory usage: 2.3 KB


In [5]:
#reading in sanitation districts
dsny = pd.read_csv('DSNY_Districts.csv')
dsny_codes = dsny[['DISTRICT', 'DISTRICTCODE']]
dsny_codes.head()

,DISTRICT,DISTRICTCODE
0,SI03,503
1,05599998 40.54236129800006,-74.14129590299996 40.54234409500003
2,SI02,502
3,02884299994 40.59100730500006,-74.20206877699997 40.59115276000006
4,SI01,501


In [6]:
#getting rid of all rows that are not decimal/numeric in DISTRICTCODE
codes = dsny_codes.loc[dsny_codes['DISTRICTCODE'].str.isdecimal()]
codes
#resetting index to count from 0
codes = codes.reset_index(drop=True)

In [19]:
codes['DISTRICTCODE_f'] = codes.DISTRICTCODE.astype(float)
codes.head()

,DISTRICT,DISTRICTCODE,DISTRICTCODE_f
0,SI03,503,503.0
1,SI02,502,502.0
2,SI01,501,501.0
3,BX10,210,210.0
4,BX02,202,202.0


In [20]:
#merging dist and codes to get  geometry for the sanitation districts
merged = pd.merge(dist, codes, left_on = 'boro_cd', right_on = 'DISTRICTCODE_f')
merged.head()

,boro_cd,shape_area,shape_leng,geometry,DISTRICT,DISTRICTCODE,DISTRICTCODE_f
0,101.0,4.268917e+07,74168.753335,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",MN01,101,101.0
1,102.0,3.768920e+07,34130.593766,"POLYGON ((-74.00779 40.74197, -74.00697 40.741...",MN02,102,102.0
2,103.0,4.687976e+07,30468.302497,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",MN03,103,103.0
3,104.0,4.931180e+07,67623.951686,"POLYGON ((-73.99557 40.77387, -73.99394 40.773...",MN04,104,104.0
4,105.0,4.379030e+07,35288.304590,"POLYGON ((-73.98149 40.76788, -73.97908 40.766...",MN05,105,105.0


In [21]:
#reading in file for monthly recycling tonnage data
tonnage = pd.read_csv('https://data.cityofnewyork.us/resource/ebb7-mvp5.csv')
tonnage.info()
#using the api only gives us 1000 of like 27,000 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   month                1000 non-null   object 
 1   borough              1000 non-null   object 
 2   communitydistrict    1000 non-null   object 
 3   refusetonscollected  997 non-null    float64
 4   papertonscollected   997 non-null    float64
 5   mgptonscollected     997 non-null    float64
 6   resorganicstons      103 non-null    float64
 7   schoolorganictons    18 non-null     float64
 8   leavesorganictons    22 non-null     float64
 9   xmastreetons         60 non-null     float64
 10  borough_id           997 non-null    float64
dtypes: float64(8), object(3)
memory usage: 86.1+ KB


In [22]:
def cd_codes(boro):
    if boro == 'Bronx':
        val = 'BX'
    elif boro == 'Brooklyn':
        val = 'BK'
    elif boro == 'Queens':
        val = 'Q'
    elif boro == 'Manhattan':
        val = 'MN'
    else:
        val = 'SI'
    return val
tonnage['cd_code'] = ''
tonnage['cd_code'] = tonnage.apply(lambda x: cd_codes(x['borough']), axis = 1)

#tonnage.head(50)
tonnage['cd'] = tonnage['cd_code'] + tonnage['communitydistrict']
tonnage.head(50)
#set index to cd
tonnage.set_index('cd', inplace = True)
tonnage.head(50)

,month,borough,communitydistrict,refusetonscollected,papertonscollected,mgptonscollected,resorganicstons,schoolorganictons,leavesorganictons,xmastreetons,borough_id,cd_code
cd,,,,,,,,,,,,
BX01,2021 / 07,Bronx,01,496.01,25.05,28.90,NaN,NaN,NaN,NaN,2.0,BX
BX02,2021 / 07,Bronx,02,413.81,33.12,38.15,NaN,NaN,NaN,NaN,2.0,BX
BX03,2021 / 07,Bronx,03,491.32,37.15,51.37,NaN,NaN,NaN,NaN,2.0,BX
BX04,2021 / 07,Bronx,04,970.41,57.11,72.22,NaN,NaN,NaN,NaN,2.0,BX
BX05,2021 / 07,Bronx,05,867.76,56.87,75.28,NaN,NaN,NaN,NaN,2.0,BX
BX06,2021 / 07,Bronx,06,512.17,59.13,67.57,NaN,NaN,NaN,NaN,2.0,BX
BX07,2021 / 07,Bronx,07,858.11,70.66,134.50,NaN,NaN,NaN,NaN,2.0,BX
BX08,2021 / 07,Bronx,08,601.53,80.84,77.28,NaN,NaN,NaN,NaN,2.0,BX
BX09,2021 / 07,Bronx,09,957.13,62.07,90.83,NaN,NaN,NaN,NaN,2.0,BX


In [23]:
def reform(cd):
    if 'S' in cd[2] or 'N' in cd[2]:
        new = cd[:2] + cd[3:]
    elif 'E' in cd[1] or 'W' in cd[1]:
        new = cd[:1] + cd[2:]
    else:
        new = cd
    return new

merged['re_codes'] = merged.apply(lambda x: reform(x['DISTRICT']), axis = 1)
merged.head(20)

,boro_cd,shape_area,shape_leng,geometry,DISTRICT,DISTRICTCODE,DISTRICTCODE_f,re_codes
0,101.0,4.268917e+07,74168.753335,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",MN01,101,101.0,MN01
1,102.0,3.768920e+07,34130.593766,"POLYGON ((-74.00779 40.74197, -74.00697 40.741...",MN02,102,102.0,MN02
2,103.0,4.687976e+07,30468.302497,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",MN03,103,103.0,MN03
3,104.0,4.931180e+07,67623.951686,"POLYGON ((-73.99557 40.77387, -73.99394 40.773...",MN04,104,104.0,MN04
4,105.0,4.379030e+07,35288.304590,"POLYGON ((-73.98149 40.76788, -73.97908 40.766...",MN05,105,105.0,MN05
5,106.0,3.873365e+07,43163.835799,"MULTIPOLYGON (((-73.96128 40.73016, -73.96128 ...",MN06,106,106.0,MN06
6,107.0,5.315282e+07,39863.701364,"POLYGON ((-73.96838 40.78660, -73.96885 40.785...",MN07,107,107.0,MN07
7,108.0,5.517244e+07,53927.099287,"MULTIPOLYGON (((-73.93557 40.77192, -73.93566 ...",MN08,108,108.0,MN08
8,309.0,4.532633e+07,29944.474194,"POLYGON ((-73.94508 40.65572, -73.94716 40.655...",BKN09,309,309.0,BK09
9,109.0,4.189803e+07,34958.004577,"POLYGON ((-73.94020 40.82865, -73.94154 40.826...",MN09,109,109.0,MN09


In [24]:
#getting the mean for all
ref = tonnage.groupby('cd')['refusetonscollected'].mean()
san_dist = merged[['re_codes', 'geometry']].set_index('re_codes')
san_dist.info()
ref.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 59 entries, MN01 to SI03
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  59 non-null     geometry
dtypes: geometry(1)
memory usage: 944.0+ bytes


cd
BK01    5398.843529
BK02    2425.151765
BK03    4219.329412
BK04    3292.947059
BK05    4878.291765
Name: refusetonscollected, dtype: float64

In [13]:
#make choropleth map to visualize tonnage within community districts
nyc = folium.Map(location = [40.7128, -74.0060], tiles = 'cartodbpositron', zoom_start = 10)
Choropleth(geo_data = san_dist.__geo_interface__,
          data = ref,
          key_on = 'feature.id',
          fill_color = 'YlGnBu',
          legend_name = 'Average Total Refuse Collected in Tons by Sanitation District 2020 - 2021').add_to(nyc)
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width = '100%', height = '500px')
embed_map(nyc, 'nyc.html')

In [14]:
#splitting month column into month, year columns
tonnage['month']
tonnage[['year','mon']] = tonnage.month.str.split(" /",expand=True)
tonnage

,month,borough,communitydistrict,refusetonscollected,papertonscollected,mgptonscollected,resorganicstons,schoolorganictons,leavesorganictons,xmastreetons,borough_id,cd_code,year,mon
cd,,,,,,,,,,,,,,
BX01,2021 / 07,Bronx,01,496.01,25.05,28.90,NaN,NaN,NaN,NaN,2.0,BX,2021,07
BX02,2021 / 07,Bronx,02,413.81,33.12,38.15,NaN,NaN,NaN,NaN,2.0,BX,2021,07
BX03,2021 / 07,Bronx,03,491.32,37.15,51.37,NaN,NaN,NaN,NaN,2.0,BX,2021,07
BX04,2021 / 07,Bronx,04,970.41,57.11,72.22,NaN,NaN,NaN,NaN,2.0,BX,2021,07
BX05,2021 / 07,Bronx,05,867.76,56.87,75.28,NaN,NaN,NaN,NaN,2.0,BX,2021,07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q07,2020 / 03,Queens,07,6163.72,788.01,785.12,104.57,NaN,NaN,NaN,4.0,Q,2020,03
Q08,2020 / 03,Queens,08,3454.12,422.73,417.58,102.89,NaN,NaN,NaN,4.0,Q,2020,03
Q09,2020 / 03,Queens,09,3613.19,423.53,556.55,158.20,21.61,NaN,NaN,4.0,Q,2020,03


In [15]:
#getting only 2020 refuse
tonnage['year'] = tonnage['year'].astype(str).astype(int)
#tonnage['year']
ref_2020 = tonnage[tonnage.year == 2020]
ref_2020

,month,borough,communitydistrict,refusetonscollected,papertonscollected,mgptonscollected,resorganicstons,schoolorganictons,leavesorganictons,xmastreetons,borough_id,cd_code,year,mon
cd,,,,,,,,,,,,,,
BX01,2020 / 12,Bronx,01,2275.73,177.02,181.35,NaN,NaN,NaN,NaN,2.0,BX,2020,12
BX02,2020 / 12,Bronx,02,1723.33,163.03,188.08,NaN,NaN,4.5,NaN,2.0,BX,2020,12
BX03,2020 / 12,Bronx,03,2281.47,225.45,259.82,NaN,NaN,NaN,NaN,2.0,BX,2020,12
BX04,2020 / 12,Bronx,04,4395.54,339.69,405.88,NaN,NaN,NaN,NaN,2.0,BX,2020,12
BX05,2020 / 12,Bronx,05,3740.06,311.92,398.73,NaN,NaN,NaN,NaN,2.0,BX,2020,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q07,2020 / 03,Queens,07,6163.72,788.01,785.12,104.57,NaN,NaN,NaN,4.0,Q,2020,03
Q08,2020 / 03,Queens,08,3454.12,422.73,417.58,102.89,NaN,NaN,NaN,4.0,Q,2020,03
Q09,2020 / 03,Queens,09,3613.19,423.53,556.55,158.20,21.61,NaN,NaN,4.0,Q,2020,03


In [16]:
ref_2020

,month,borough,communitydistrict,refusetonscollected,papertonscollected,mgptonscollected,resorganicstons,schoolorganictons,leavesorganictons,xmastreetons,borough_id,cd_code,year,mon
cd,,,,,,,,,,,,,,
BX01,2020 / 12,Bronx,01,2275.73,177.02,181.35,NaN,NaN,NaN,NaN,2.0,BX,2020,12
BX02,2020 / 12,Bronx,02,1723.33,163.03,188.08,NaN,NaN,4.5,NaN,2.0,BX,2020,12
BX03,2020 / 12,Bronx,03,2281.47,225.45,259.82,NaN,NaN,NaN,NaN,2.0,BX,2020,12
BX04,2020 / 12,Bronx,04,4395.54,339.69,405.88,NaN,NaN,NaN,NaN,2.0,BX,2020,12
BX05,2020 / 12,Bronx,05,3740.06,311.92,398.73,NaN,NaN,NaN,NaN,2.0,BX,2020,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q07,2020 / 03,Queens,07,6163.72,788.01,785.12,104.57,NaN,NaN,NaN,4.0,Q,2020,03
Q08,2020 / 03,Queens,08,3454.12,422.73,417.58,102.89,NaN,NaN,NaN,4.0,Q,2020,03
Q09,2020 / 03,Queens,09,3613.19,423.53,556.55,158.20,21.61,NaN,NaN,4.0,Q,2020,03


In [17]:
#creating map for 2020
nyc_2020 = folium.Map(location = [40.7128, -74.0060], tiles = 'cartodbpositron', zoom_start = 10)
Choropleth(geo_data = san_dist.__geo_interface__,
          data = ref_2020['refusetonscollected'],
          key_on = 'feature.id',
          fill_color = 'YlGnBu',
          legend_name = 'Total Refuse Collected in Tons by Sanitation District for 2020').add_to(nyc_2020)

def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width = '100%', height = '500px')
embed_map(nyc_2020, 'nyc2020.html')